In [10]:
!pip install twython

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style('whitegrid')

In [2]:
from twython import Twython

In [3]:
access_token = "761441357315440640-suCCQJo6kuufi3PmcYUl2y9kNyYb8C0"
access_token_secret = "nN4nX0LhlUZHN31LLYU1neOxg7elvb4LIo9KkX7gMDMaN"
api_key = "oMlZlYVi6MerYj7SZzcYWvgVr"
api_secret_key = "OW8cYRS69LUQ1gD5rKULGi4QtuBoj0OX5hRyJI5HVBbzTLZzam"

In [4]:
#https://developer.twitter.com/en/docs/twitter-api/v1/tweets/search/api-reference/get-search-tweets
#https://developer.twitter.com/en/docs/tweets/timelines/guides/working-with-timelines
#count max is 100
def get_tweets(query='', pagestocollect = 20, count=100, result_type='mixed', since=None, until=None, geocode=None):
    # This method downloads the tweets from twitter and returns a list of dictionaries
    # each representing a tweet.
    # This function was inspired from: http://www.hristogueorguiev.com/
    # basic-twitter-data-miner-and-data-analysis-python-twython-twitter-api-pandas-matplotlib/
    # The get_tweets function is based on the Standard search API form Twitter

    # AUTHENTICATE
    twitter_obj = Twython(api_key, api_secret_key, access_token, access_token_secret)

    results = twitter_obj.search(q=query, include_entities='true',
                             tweet_mode='extended',count=count,
                             result_type=result_type,lang='en',
                             since=since, 
                             until=until,
                             geocode=geocode)

# The following code follows this logic
# https://developer.twitter.com/en/docs/tweets/timelines/guides/working-with-timelines
    data = results['statuses']
    i=0
    ratelimit=1

    while results['statuses'] and i<pagestocollect: 

        if ratelimit < 1: 
            # Rate limit time out needs to be added here in order to
            # collect data exceeding available rate-limit 
            print(str(ratelimit)+'Rate limit!')
            break

        mid= results['statuses'][len(results['statuses']) -1]['id']-1

        results = twitter_obj.search(q=query, max_id=str(mid)
                             ,include_entities='true',
                             tweet_mode='extended',count=count,
                             result_type=result_type,lang='en',
                             since=since,
                             until=until,
                             geocode=geocode)

        data+=results['statuses']
        i+=1
        ratelimit = int(twitter_obj.get_lastfunction_header('x-rate-limit-remaining'))

    return data

In [37]:
#geocode="43.653226,-79.383184,100km" #Toronto
#geocode="49.525238,-93.874023,4000km" #North America
tweet_data = get_tweets(query='Walmart',result_type='recent', geocode="49.525238,-93.874023,4000km")

In [38]:
df = pd.DataFrame()
df['user'] = list(map(lambda tweet: tweet['user']['screen_name'], tweet_data))
df['time'] = list(map(lambda tweet: tweet['created_at'], tweet_data))
df['tweet_text'] = list(map(lambda tweet: tweet['full_text'], tweet_data))
df['lat'] = list(map(lambda tweet: tweet['geo']['coordinates'][0] if tweet['geo'] != None and tweet['geo']['coordinates'][0] != 0.0 else None, tweet_data))
df['long'] = list(map(lambda tweet: tweet['geo']['coordinates'][1] if tweet['geo'] != None and tweet['geo']['coordinates'][0] != 0.0 else None, tweet_data))
df['location'] = list(map(lambda tweet: tweet['user']['location'], tweet_data))
df['hashtags'] = list(map(lambda tweet: tweet['entities']['hashtags'], tweet_data)) 

In [39]:
def find_search_term(tweet):
    matches = []
    for i, term in enumerate(search_terms):
        if(term.lower() in tweet.lower()):
            matches.append(term)
    return matches

In [40]:
search_terms = ['Walmart']
df['search_term'] = df['tweet_text'].apply(lambda tweet: find_search_term(tweet))

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2093 entries, 0 to 2092
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   user         2093 non-null   object 
 1   time         2093 non-null   object 
 2   tweet_text   2093 non-null   object 
 3   lat          3 non-null      float64
 4   long         3 non-null      float64
 5   location     2093 non-null   object 
 6   hashtags     2093 non-null   object 
 7   search_term  2093 non-null   object 
dtypes: float64(2), object(6)
memory usage: 130.9+ KB


In [42]:
pd.set_option('display.max_rows', 999)
pd.set_option('display.max_columns', 999)
df.head(400)

,user,time,tweet_text,lat,long,location,hashtags,search_term
0,BryanxD_98,Mon Nov 16 19:21:25 +0000 2020,RT @Lbabinz: @WalmartCAGaming Any news by chan...,NaN,NaN,,[],[Walmart]
1,rfd_hotdeals,Mon Nov 16 19:18:47 +0000 2020,"[Walmart] Black Friday deal Acer Nitro 15.6"" N...",NaN,NaN,Canada,[],[Walmart]
2,arejayplays,Mon Nov 16 19:16:58 +0000 2020,@JermainePlays Bro it stings so bad. I carted...,NaN,NaN,"Virginia, USA",[],[Walmart]
3,VoteForChanges,Mon Nov 16 19:16:14 +0000 2020,RT @QasimRashid: COVID19 has closed 20% of sma...,NaN,NaN,,[],[Walmart]
4,CulpeperSE,Mon Nov 16 19:10:07 +0000 2020,Andrew Zachary Ray resolved his theft charge i...,NaN,NaN,"Virginia, USA",[],[Walmart]
5,sunnybama,Mon Nov 16 19:07:23 +0000 2020,RT @QasimRashid: COVID19 has closed 20% of sma...,NaN,NaN,Safe,[],[Walmart]
6,IHaveABook4U,Mon Nov 16 19:06:20 +0000 2020,RT @QasimRashid: COVID19 has closed 20% of sma...,NaN,NaN,,[],[Walmart]
7,thebeccamartin,Mon Nov 16 19:02:58 +0000 2020,Walmart has really dropped my hours from 37.50...,NaN,NaN,Canada,[],[Walmart]
8,RENXca,Mon Nov 16 18:55:00 +0000 2020,Walmart Inc. is selling a majority stake in Ja...,NaN,NaN,Canada,"[{'text': 'RetailNews', 'indices': [212, 223]}...",[Walmart]
9,ASkillman14,Mon Nov 16 18:54:12 +0000 2020,RT @QasimRashid: COVID19 has closed 20% of sma...,NaN,NaN,,[],[Walmart]


In [20]:
df.to_csv('tweets.csv')

In [5]:
df = pd.read_csv('../src/data/tweets.csv')

In [6]:
df.head()

,Unnamed: 0,user,time,tweet_text,lat,long,location,hashtags
0,0,knitntherapist,Fri Nov 13 01:22:43 +0000 2020,@CZEdwards I can’t be the only one with vision...,NaN,NaN,North of Toronto,[]
1,1,the_james_the,Fri Nov 13 01:22:08 +0000 2020,"I love Jeopardy as the absolute, polar opposit...",NaN,NaN,"Toronto, Ontario",[]
2,2,shesconnected,Fri Nov 13 01:21:06 +0000 2020,@TangerineBank Love my VISA debit card. You ca...,NaN,NaN,"Toronto, Canada","[{'text': 'MoneySavvy', 'indices': [178, 189]}]"
3,3,BoltonBrewer,Fri Nov 13 01:20:34 +0000 2020,RT @CPA_Ontario: Keep your phone charged at al...,NaN,NaN,"Bolton, ON, Canada.",[]
4,4,PrimaryCareRD,Fri Nov 13 01:19:14 +0000 2020,RT @drandrewb: you can't say you respect your ...,NaN,NaN,NaN,[]


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5089 entries, 0 to 5088
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  5089 non-null   int64  
 1   user        5089 non-null   object 
 2   time        5089 non-null   object 
 3   tweet_text  5089 non-null   object 
 4   lat         2 non-null      float64
 5   long        2 non-null      float64
 6   location    4038 non-null   object 
 7   hashtags    5089 non-null   object 
dtypes: float64(2), int64(1), object(5)
memory usage: 318.2+ KB
